<a href="https://colab.research.google.com/github/synfronteras/drylablessons/blob/main/PraticaClubeSynfronteras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FUNDAMENTOS EM MODELAGEM DE CIRCUITOS GENÉTICOS**
Produzido e organizado por: Samuel Chagas de Assis


# 1. **Slides de Introdução**
Podem ser acessados [aqui](https://docs.google.com/presentation/d/1OJLfBi74SeuaQeNF2k3sr1ZHO31J1rVEuam4yliw4Bg/edit?usp=sharing)

# 2. **Referências Importantes**

Fluxo de atividades e parte dos códigos foram adaptados de [Elowitz & Bois (2020)](http://be150.caltech.edu/2020/content/lessons/03_small_circuits.html)

Código do *Genetic Toggle Switch* foi adaptado do repositório de [Ms. Felipe Buson](https://github.com/fxbuson/PSB_Assignment2)

*Referências de estudos*:

Alon, U. An Introduction to Systems Biology: Design Principles of Biological Circuits. Taylor & Francis, Second Edition, 2020.

Ingalls, B. P. Mathematical Modeling in Systems Biology: An Introduction. The MIT Press, 2013. 




# 2. **Introdução ao Colab**




In [ ]:
#Bora brincar


In [ ]:
#Bibliotecas
import numpy as np
import scipy.integrate
import scipy.optimize

import bokeh.io
import bokeh.plotting #import ColumnDataSource, figure, show
from bokeh.models import Slider, Select, TextInput, MultiSelect, RangeSlider, CustomJS
from bokeh.layouts import column, row

import colorcet

bokeh.io.output_notebook()

colors = bokeh.palettes.Reds9

## **1. *Dynamics*: estabilidade proteíca e tempo de resposta**

---

**Pergunta**: Quanto tempo até atingir determinado nível de concentração da proteína de interesse? Podemos diminuir o tempo de resposta?

**Pense**: Esse nível serveria para qual função/aplicação? (ex: lisar um patógeno, sinalização de uma detecção, etc)

**Comece com**: o Básico

<br>

<div>
   <img src="https://drive.google.com/uc?export=view&id=1x-BfxEig33m5M7mjYQSfjE2gOGDsLwHe" width="400">
</div>

<br>

>$DNA \xrightarrow{\text{$β_m$}}\ Proteína \xrightarrow{\text{$γ_p$}}\ ∘deg$


Para fins didáticos, vamos iniciar com a expressão gênica e negligenciar o nível de mRNA (podemos dizer também: assumir que Bm já representa a taxa de transcrição). Com isso, temos para um gene expresso repentinamente: 

<br>

$\frac{dx}{dt} = β  - γx$ 

<br>

$x_{ss} = \frac{β}{γ}$ (Estado quando o comportamento do sistema permanece constante)


<br>

$x(t) = \frac{β}{γ}(1  - e^{-γt})$ 
(função arbitrária, assumida nessa atividade, onde a produção "sempre" estará em $x_{ss}$(*steady state*) e a taxa de remoção é uma função logarítimica com base $e$)

<br>

*Premissas*:

* tempo de resposta: $γ^{-1}$ para atingir uma concentração ao fator $1-e^{-1}$ do *steady-state*. 



In [ ]:
#@title Produção vs Degradação/Diluição {display-mode: "form"}

#Barras das Variáveis
beta_slider = Slider(start=0.1, end=120, value=116, step=.1, title="Beta (Bm): Produção (Transcrição+Tradução)")
gamma_slider = Slider(start=0.1, end=10, value=5, step=.1, title="Gamma (y): Instabilidade Proteica (Degradação+Diluição)")

# Dynamics
x = np.linspace(0, 6, 400)
y = beta_slider.value / gamma_slider.value * (1 - np.exp(-gamma_slider.value * x))

# Para marcar o tempo de resposta (quando atingimos 1-1/e)
x0 = 1 / gamma_slider.value
y0 = beta_slider.value / gamma_slider.value * (1 - np.exp(-1))

source = bokeh.models.ColumnDataSource(data=dict(x=x, y=y))

# Gráfico de Produção
p = bokeh.plotting.figure(
    frame_height=300,
    frame_width=600,
    x_axis_label="t",
    y_axis_label="x(t)",
    #title=f"β = {beta}, γ = {gamma}",
)
p.line('x', 'y', source = source, color = colors[5], line_width=2)

# Adicionando as legendas e marcações

source1 = bokeh.models.ColumnDataSource(
    data=dict(x0=[x0], y0=[y0], text=["response time"])
)
p.circle(x="x0", y="y0", color = colors[5], source=source1, size=10)
p.line(x='x0', y = 5, color = colors[5], source = source1)
p.add_layout(
    bokeh.models.LabelSet(
        x="x0", y="y0", text="text", source=source1, x_offset=10, y_offset=-10
    )
)
span = bokeh.models.Span(location=x0, 
                         level="underlay", 
                         dimension="height", 
                         line_color="black",
                         line_dash="dashed",
    )

p.add_layout(span)


callback = CustomJS(args=dict(source=source, source1=source1, span = span, beta=beta_slider, gamma=gamma_slider),
                    code="""
    span.location = 1 / cb_obj.value
    const data = source.data;
    const data1 = source1.data;
    const Bm = beta.value;
    const Py = gamma.value;
    const x0 = data1['x0']
    const y0 = data1['y0']
    const x = data['x']
    const y = data['y']
    for (let i = 0; i < x.length; i++) {
        y[i] = Bm/Py*(1-Math.exp(-Py*x[i]));
        x0[i] = 1/Py;
        y0[i] = Bm/Py*(1-Math.exp(-1));
        
    }
    source.change.emit();
    source1.change.emit();
""")

beta_slider.js_on_change('value', callback)
gamma_slider.js_on_change('value', callback)

layout = row(
    p,
    column(beta_slider, gamma_slider),
)

bokeh.io.show(layout)

In [ ]:

#@title Ajustando Parâmetros {display-mode: "form"}
#@markdown **Definir Taxa de Produção e Degradação.**


#Taxa de Produção
beta = 116  #@param {type: "slider", min: 50, max: 150}

#Taxa de Decaimento/Degrad
γ1 = 1  #@param {type: "slider", min: 1, max: 7}
γ2 = 3  #@param {type: "slider", min: 1, max: 7}
γ3 = 7 #@param {type: "slider", min: 1, max: 7}


#@markdown ---

In [ ]:
#@title Produção vs Degradação/Diluição {display-mode: "form"}

# Parâmetros
gamma = np.array([γ1, γ2, γ3])

# Ajustando Dinâmica
x = np.linspace(0, 6, 400)
y = [beta / g * (1 - np.exp(-g * x)) for g in gamma]

# Configurando Gráficos
p1 = bokeh.plotting.figure(
    frame_height=300,
    width=800,
    x_axis_label="t",
    y_axis_label="x(t)",
    title=f"β₁ = {beta}, γ = {gamma}",
)
p2 = bokeh.plotting.figure(
    frame_height=300,
    width=800,
    x_axis_label="t",
    y_axis_label="x(t)",
    title="Mesmo Gráfico, normalizado por steady states",
)
p2.x_range = p1.x_range

# Plottando
for y_vals, g, color in zip(y, gamma, colors):
    p1.line(x, y_vals, color=color, line_width=2)
    p1.circle(1 / g, beta / g * (1 - np.exp(-1)), color=color, size=10)
    p2.line(x, y_vals / y_vals.max(), color=color, line_width=2)
    p2.circle(1 / g, 1 - np.exp(-1), color=color, size=10)

bokeh.io.show(bokeh.layouts.gridplot([p1, p2], ncols=2))

### **Conclusão**
---
 **Princípio de Design:** A partir do ajuste de síntese e estabilidade proteíca, o aumento do ciclo de produção e remoção diminui o tempo de reposta (acelera a resposta) de um sistema de expressão gênica.


## **2. Autoregulação Negativa**

---

**Pergunta**: Mas se a proteína (ex: fatores transcricionais) forem estáveis, como os organismos naturalmente controlam as expressões/repressões de forma rápida? 

**Pense**: Muitos sistemas biológicos são naturalmente auto-controlados, muitos sistemas precisam se autosustentar mediante a perturbações externas. Com isso, existem alguns mecanismos chamados de "*network motif*" (intra-sistemas?) que executam determinada função para garantir estabilidade, como:

<br>

*Negative Autoregulatory Loop* 

<div>
   <img src="https://drive.google.com/uc?export=view&id=1XUw_oJNz9M6EJXNdOny2tYg3DXJaSpQa" width="400">
</div>

<br>


Portanto, ....

<br>

$\frac{dx}{dt} = \frac{β}{1+\frac{x}{K_d}} - γx$ 

<br>

$\frac{β}{1+\frac{x}{K_d}}$ (Termo de síntesede $x$ baseada na repressão do próprio $x$)

<br>

$K_d$ = Constante de dissociação $\frac{k_-}{k_+}$

<br>


*Premissas*:

* Sistema autoregulador "forte": ${β/γ} >> {K_d}$, em seu nível de produção máxima, possui $x$ suficiente para se autoreprimir completamente.


*beta*: taxa de expressão(produção)

*gamma*: taxa de degradação

*Kd*: Constante de dissociação



In [ ]:
#@title Ajustando Parâmetros {display-mode: "form"}
#@markdown **Definir Taxa de Produçã, Degradação e Dissociação.**


#Taxa de Produção
beta = 116  #@param {type: "slider", min: 50, max: 150}

gamma = 4  #@param {type: "slider", min: 1, max: 10}

Kd = 2  #@param {type: "slider", min: 1, max: 10}

#@markdown ---

In [ ]:
#@title Autoregulação Negativa {display-mode: "form"}

# Negative autoregulation right hand side
def neg_auto_reg_rhs(x, t, beta, Kd, gamma):
    return beta / (1 + x / Kd) - gamma * x

# Parameters
t = np.linspace(0, 6, 400)

# Negative autoregulated solution
y = scipy.integrate.odeint(neg_auto_reg_rhs, 0, t, args=(beta, Kd, gamma))
nar = y[:, 0]

# Unregulated solution
unreg = 1 - np.exp(-gamma * t)

# Create plots of unnormalized responses
p = [
    bokeh.plotting.figure(
        frame_height=300, width=850, x_axis_label="t", y_axis_label="x(t)"
    )
    for _ in [0, 1]
]
p[0].x_range = p[1].x_range

p[0].title.text = "Dinâmica de ativação ± autoregulação negativa"

p[0].line(
    t, 
    nar, 
    line_width=2, 
    color = colors[1],
    legend_label="Autoregulação Negativa")

p[0].line(
    t,
    beta / gamma * unreg,
    line_width=2,
    color="grey",
    legend_label="Sem Regulação",
)
p[0].legend.location = "center_right"

# Create plot of normalized responses
p[1].title.text = "Mesmo Gráfico, normalizado"
p[1].line(t, nar / nar.max(), color = colors[1], line_width=2)
p[1].line(t, unreg, line_width=2, color="grey")

bokeh.io.show(bokeh.layouts.gridplot(p, ncols=2))

### **Conclusão**
---
 **Princípio de Design:**
* Diminuição no level de expressão em *steady state*
* Diminuição do tempo de resposta

<br>
<div>
   <img src="https://drive.google.com/uc?export=view&id=1Z2yLcdLlIdBYrNUIM34hiKykkHaIjK0O" width="400">
</div>

Fonte: [Rosenfeld et al., J. Mol. Biol., 2002](https://doi.org/10.1016/S0022-2836(02)00994-4)

<br>

* Redução da variabilidade estocástica célula-célulca ("noise", perturbações)

<div>
   <img src="https://drive.google.com/uc?export=view&id=1thEFEElNoEJPl-0YEPN2N3B1z_s3NBNg" width="500">
</div>

Fonte: [Becskei and Serrano, Nature, 2000](https://www.nature.com/articles/35014651)









## **3. Função de Hill e Ultrasensitividade**

---

Função de Hill contribui para analisar sistemas que possuem interações de cooperatividade, ou performam um comportamento ultrasensitivo. Essas característica está relacionada com o potencialde alguns efetores se ligarem a um sítio e aumentarem a sua afinidade a outros fatores iguais. Esse tipo de interação pode ter um pequeno efeito inicialmente, mas de forma repentina (acúmulo de efetores) ter um efeito grande. 

<br>

<div>
   <img src="https://drive.google.com/uc?export=view&id=1xh5d03GffbBL-oPUfmRgjZgcrCcWyJUk" width="400">
</div>

Fonte: [Giorgetti, L. et al., 2010](https://doi.org/10.1016/j.molcel.2010.01.016)

<br>

*Função de ativação*: $f_{act}(x) = \frac{x^n}{k^n + x^n} = \frac{(x/k)^n}{1 + (x/k)^n}$

<br>

*Função de repressão*: $f_{rep}(x) = \frac{k^n}{k^n + x^n} = \frac{1}{1 + (x/k)^n}$


Nessas expressões, $k$ representa a concentração da metade do valor máximo, e $n$ é o coeficiente de Hill. Esse coeficiente quantifica quanto ultrasensitivo a resposta será (limite de n = ∞ $\rightarrow$ *step function*) 

In [ ]:
#@title Ajustando Parâmetros {display-mode: "form"}
#@markdown **Definir Coeficiente de Hill e Concentração Máxima.**


#Taxa de Produção
max_conc = 302  #@param {type: "slider", min: 100, max: 800}

#Taxa de Decaimento/Degrad
K = 128  #@param {type: "slider", min: 1, max: 400}
n1 = 1  #@param {type: "slider", min: 1, max: 10, step: 0.1}
n2 = 7.8  #@param {type: "slider", min: 1, max: 10, step: 0.1}
n3 = 6  #@param {type: "slider", min: 1, max: 10, step: 0.1}



#@markdown ---

In [ ]:
#@title Função de Hill e Ultrasensitividade {display-mode: "form"}

list_n = [n1, n2, n3, 12]
x = np.linspace(1, max_conc, 1000)
f = [(x/K)**n / (1 + (x/K)**n) for n in list_n]

# Build plot
p = bokeh.plotting.figure(
    height=275,
    width=700,
    x_axis_label="x",
    y_axis_label="Ativando a Função de Hill",
#    x_range=[x[0], x[-1]],
    x_axis_type="log"
)

for f_act, n, color in zip(f, list_n, colors):

    p.line(x, f_act, line_width=2, color=color, legend_label=f"n = {n}")
p.legend.location = "center_right"
bokeh.io.show(p)

## **4. Autoregulação Positiva**

---

**Pergunta**: E se fosse necessário dois estados (ligado/desligado), como um circuito autoregulador funcionaria? 

**Pense**: Como seria o estado de *ligado* e *desligado*? E quais interações seriam necessárias?

<br>

*Positive Autoregulatory Loop* 

<div>
   <img src="https://drive.google.com/uc?export=view&id=1Ksb0B7m2jwIwkPeQPqHrLtfuh-qOu9aZ" width="400">
</div>

<br>

<br>

$\frac{dx}{dt} = \frac{βx^n}{x^n+k^n} - γx$ 

<br>

$\frac{βx^n}{x^n+k^n}$ (Termo de síntesede $x$ baseada na repressão do próprio $x$)

<br>

$k$ =  concentração da metade do valor máximo (ex: $IC_{50}$)
<br>

**No Gráfico:** Quando a taxa de produção = taxa de degradação, temos os *fixed points* . Entre esses pontos é possível avaliar a estabilidade. Quando estáveis (círculos pretos) e instaveis (círculos brancos), podem distiguir dois estados do sistema. Por exemplo: dois pontos estaveis e um *fixed point* instável =  biestabilidade. 

No entanto, autoregulaçãopositiva não é suficiente para garantir biestabilidade, sendo necessário uma resposta ultrasensitiva (função de Hill)

**Tarefa**: Observar o comportamento da curva da taxa de produção em relação a taxa de remoção a partir da mudança das variáveis.

*beta*: taxa de expressão da proteína

*gamma*: taxa de degradação+diluição (remoção)

*K*: concentração na metade do valor máximo de ativação.

*n*: coeficiente de Hill (cooperatividade)



In [ ]:
#@title Ajustando Parâmetros {display-mode: "form"}
#@markdown **Definir Taxa de Produção e Degradação.**


#Taxa de Produção
beta = 10  #@param {type: "slider", min: 1, max: 10}

#Taxa de Decaimento/Degrad
gamma = 1.2  #@param {type: "slider", min: 1, max: 3, step: 0.1}

K = 4  #@param {type: "slider", min: 1, max: 5}
n = 5  #@param {type: "slider", min: 1, max: 5, step: 0.1}


#@markdown ---

In [ ]:
#@title Biestabilidade {display-mode: "form"}

# Theroetical curves
x = np.linspace(0, 20, 400)
prod = beta * x ** n / (x ** n + K ** n)
removal = gamma * x

# Fixed points
fp_rhs = lambda x: beta / gamma * x ** n / (x ** n + K ** n)

fixed_points = [0,
                  float(scipy.optimize.fixed_point(fp_rhs, K)),
                  float(scipy.optimize.fixed_point(fp_rhs, 5)),]

# Build plot
p = bokeh.plotting.figure(
    frame_height=275,
    frame_width=675,
    x_axis_label="x",
    y_axis_label="production or removal rate",
    title=f"β = {beta}, γ = {gamma} , K = {K}, n = {n}",
)

# Plot production and removal rates
p.line(x, prod, line_width=2, color="#1f77b4")
p.line(x, removal, line_width=2, color="orange")

# Plot fixed points
for i, fp in enumerate(fixed_points):
    p.add_layout(
        bokeh.models.Span(
            location=fp,
            level="underlay",
            dimension="height",
            line_color="black",
            line_dash="dashed",
        )
    )
    fill_color = "white" if i % 2 else "black"
    p.circle(
        [fp],
        [gamma * fp],
        color="black",
        size=10,
        line_width=2,
        fill_color=fill_color,
    )

# Annotate

p.text(
    x=[14],
    y=[9.933],
    text=["production rate"],
    text_color="#1f77b4",
    text_font_size="10pt",
    text_align="left",
    text_baseline="top",
)
p.text(
    x=[14],
    y=[14],
    text=["removal rate"],
    text_color="orange",
    text_font_size="10pt",
    text_align="left",
    angle=0.575,
)

bokeh.io.show(p)

### **Conclusão**
---
 **Princípio de Design:**
* Autoregulação Positiva e ultrasensitiva permite biestabilidade
* A taxa de remoção (degradação+diluição) influencia na estabilidade, como vimos anteriormente...



## **5. *Genetic Toggle Switch (Um Interruptor Genético)***

---

**Pergunta**: E como poderíamos controlar dois estados de expressão gênica? Por exemplo, em um momento produzir uma proteína **verde** e a partir de um *click*, passarmos a produzir uma proteína **vermelha**

Normalmente, em sistemas biológicos é mais comum se deparar com sistemas *positive/negative feedback* (ou retroalimentação positiva/negativa) e não *autoregulatory*. Ou seja, um gene/operon/sistema regula a expressão de um segundo gene/operon/sistema e vice-versa.

<div>
   <img src="https://drive.google.com/uc?export=view&id=1YCM82h5QHqSYOeS6NR2TkHvziYNP5RGK" width="600">
</div>

Fonte: [site](https://serc.carleton.edu/earthandmind/posts/whyFLdifficult)

<br>

*Genetic Toggle Switch*

Um sistema de *feedback* foi implementado por [Gardner et al. (2000)](https://www.nature.com/articles/35002131), um dos primeiros trabalhos que reuniu partes genéticas, circuitos e modelagem (um marco para o início da biologia sintética).

<br>

**Passo 1**

Pensar num sistema de *feedback*.

<div>
   <img src="https://drive.google.com/uc?export=view&id=1nbxQjV_Mz_i0mYn75SrWNdLbX84eydBL" width="600">
</div>

<br>

**Passo 2**

Como posso controlar esse sistema? Introduzir indutores.

<div>
   <img src="https://drive.google.com/uc?export=view&id=10gDhcbOX5vDxdu8kLCxlhhuYGQKxzmr9" width="600">
</div>

Fonte: [Gardner et al. (2000)](https://www.nature.com/articles/35002131)

<br>

**Passo 3**

Seleção das partess e Modelagem do Circuito.

<div>
   <img src="https://drive.google.com/uc?export=view&id=12IjZtYruadV7afItKOJmx9MvyT7j_2IX" width="600">
</div>

Fonte: [Cameron, et al. (2014)](https://doi.org/10.1038/nrmicro3239)


<br>

$\frac{du}{dt} = \frac{α_1}{1 + V^b} - u$  $, sendo$ $V = (\frac{v}{1+(i_v/k)^n})$

<br>

$\frac{dv}{dt} = \frac{α_2}{1 + U^b} - v$ $, sendo$ $U = (\frac{u}{1+(i_u/k)^n})$

<br>

*Premissas e Observações:*

* $α_1$ e $α_2$ são parâmetros que descrevem múltiplos efeitos: ligação da RNA polimerase, eventos de transcrição.
* Características de ligação entre os indutores e repressores são com iguais, ou seja, o coeficiente de Hill é igual para ambos sistemas.
* $V$ e $U$ são interações entre indutor e repressor (parte diferente comparado ao sistema autoregulatório) 

**Tarefa:** Cada gráfico apresenta um valor de n (n1, n2, n3). Mudar as variáveis e observar se os gráficos apresentam dois estados. 

*maxSinal*: concentração do indutor colocado em cada momento. 

*n1, n2, n3*: diferentes valores do coeficiente de Hill para ligação entre repressor+indutor. 

*time*: tempo de monitoramento. 




In [ ]:
#@title Ajustando Parâmetros {display-mode: "form"}
#@markdown **Definir Taxa de Produção e Degradação.**


maxSinal = 100  #@param {type: "slider", min: 20, max: 200}
n1 = 0  #@param {type: "slider", min: 0, max: 10, step: 0.1}
n2 = 2.7  #@param {type: "slider", min: 0, max: 10, step: 0.1}
n3 = 9.1  #@param {type: "slider", min: 0, max: 10, step: 0.1}


#@markdown ---

In [ ]:
#@title Genetic Toggle Switch (Interruptor Genético)  {display-mode: "form"}
def tog_switch_round(uv, t, parameters):
    iu, iv, au, av, k, b, n = parameters
    u, v = uv
    du = -uv[0] + au / (1 + (v / (1 + iv / k)**n)**b)
    dv = -uv[1] + av / (1 + (u / (1 + iu / k)**n)**b)
    return [du, dv]

# Parameters
t = 50
timepoints = np.linspace(0, t, t+1)

#define the values for Iu and Iv in each odeint call
induction_steps = [[100,0], [0,0] ,[0,100] ,[0,0]]

#x axis for plotting should go from 0 to the # of timepoints times the # of Iu/Iv pairs
x_axis = np.linspace(0, (t+1)*len(induction_steps), (t+1)*len(induction_steps)+1)

#define remaining constants for odeint
constants = [0, 0, 10, 9, 3, 2, 2]
test_hill = [n1, n2, n3]


p = [
    bokeh.plotting.figure(
        frame_height=300, width=850, x_axis_label="t", y_axis_label="x(t)"
    )
    for _ in test_hill
]

p[0].x_range = p[1].x_range

#loop through each thing that varies in this exercise and plot as subplots
for i in range(len(test_hill)):
    tot_results = np.array([[0., 0.]])
    for step in induction_steps:
        constants[0] = step[0]
        constants[1] = step[1]
        constants[-1], constants[-2] = [test_hill[i], test_hill[i]]
        result = scipy.integrate.odeint(tog_switch_round, y0=tot_results[-1], t=timepoints, args=(constants,))
        tot_results = np.concatenate((tot_results, result))
    u = tot_results[:,0]
    v = tot_results[:,1]
   
    p[i].line(x_axis, u, line_width=2, 
           color = colors[1],
           legend_label="u")
    p[i].line(x_axis, v, line_width=2, 
           color = colors[5],
           legend_label="v")
    p[i].title.text = "Bistable switch of repressors u and v\n\
    Hill numbers: {}\n\
    Iu and Iv at each {} timepoints:\n{}".format(constants[-1], t, induction_steps)

    p[i].legend.location = "center_right"



   

#p[test_hill[0]].x_range = p[test_hill[1]].x_range




bokeh.io.show(bokeh.layouts.gridplot(p, ncols=len(test_hill)))
#bokeh.io.show(p[0])